# FFNN neural network for EMS demand predictions

This notebook contains the code for applying neural network models to smart city data <br>


In [ ]:
# GPU check:

import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
print("Show System RAM Memory: \n \n")
!cat /proc/meminfo | egrep "MemTotal"
device_lib.list_local_devices()

Show System RAM Memory: 
 

MemTotal:       26751688 kB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6192682082628457954, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8202593707315037559
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11106408156244373582
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3503078422161347117
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

Tutorial about google colab and GPU access: <br>
https://www.youtube.com/watch?v=f1UK8KPt-KU

In [ ]:
# this allows for accessing files stored in your google drive using the path "/gdrive/My Drive/"
# mounting google drive locally:

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHuucjvSCUsGfu6Qho5IDuk1RMnkQs_edQDGMPSOl9IZ9Jz14t_gVY
Mounted at /gdrive


$\textbf{Background:}$ Tensors are data structures that you can think of as multi-dimensional arrays. Tensors are represented as n-dimensional arrays of base dataypes such as a string or integer -- they provide a way to generalize vectors and matrices to higher dimensions. The shape of a Tensor defines its number of dimensions and the size of each dimension. The rank of a Tensor provides the number of dimensions. Scalars can be used to create 0-d Tensors. Vectors and lists can be used to create 1-d Tensors. Matrices can be used to create 2-d or higher rank Tensors. The shape of a Tensor provides the number of elements in each Tensor dimension.

$\textbf{Neural Networks in Tensorflow:}$ We can also define neural networks in TensorFlow. TensorFlow uses a high-level API called Keras that provides a powerful, intuitive framework for building and training deep learning models. <br> 
Tensors can flow through abstract types called $\textit{Layers}$ -- the building blocks of neural networks. Layers implement common neural networks operations, and are used to update weights, compute losses, and define inter-layer connectivity <br>
<br>
Conveniently, TensorFlow has defined a number of Layers that are commonly used in neural networks, for example a Dense. Now, instead of using a single Layer to define our simple neural network, we'll use the Sequential model from Keras and a single Dense layer to define our network. With the Sequential API, you can readily create neural networks by stacking together layers like building blocks.

# Implementation

In [ ]:
## -- Packages  -- ##

# General
import pandas as pd
import numpy as np

# Time formatting
import datetime

# Load and save data
import pickle
# progress bar
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
#import tikzplotlib as tkz

In [ ]:
##  NN libaries ##
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


## ML libraries ##
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler

### Load data

In [ ]:
#load emergency data all of 2019
filename = '/gdrive/My Drive/Colab Notebooks/emergency_dispatches_bronx_H'
infile = open(filename,'rb')
emergency_ts_H = pickle.load(infile)
infile.close()

In [ ]:
emergency_ts_H.shape

(8760,)

### Preprocessing

In [ ]:
## Remove last 15 days of data since they are either erroneous or part of the test set to the robustness checks
emergency_ts_final = emergency_ts_H.iloc[0:-360]
emergency_ts_final.shape

(8400,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags


In [ ]:
## Split data in train and test set
train = emergency_ts_final[0:-n_test]
test = emergency_ts_final[-n_test:]
print(train.shape)
print(test.shape)

(8376,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
df.head()

,t-60,t-59,t-58,t-57,t-56,t-55,t-54,t-53,t-52,t-51,t-50,t-49,t-48,t-47,t-46,t-45,t-44,t-43,t-42,t-41,t-40,t-39,t-38,t-37,t-36,t-35,t-34,t-33,t-32,t-31,t-30,t-29,t-28,t-27,t-26,t-25,t-24,t-23,t-22,t-21,...,t-16,t-15,t-14,t-13,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t+0,t+1,t+2,t+3,t+4,t+5,t+6,t+7,t+8,t+9,t+10,t+11,t+12,t+13,t+14,t+15,t+16,t+17,t+18,t+19,t+20,t+21,t+22,t+23
FIRST_ASSIGNMENT_DATETIME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-03 12:00:00,41.0,48.0,36.0,55.0,48.0,27.0,47.0,46.0,58.0,51.0,42.0,50.0,62.0,44.0,47.0,58.0,44.0,53.0,44.0,43.0,44.0,38.0,45.0,35.0,34.0,38.0,26.0,23.0,17.0,21.0,27.0,39.0,37.0,53.0,67.0,50.0,69.0,59.0,66.0,50.0,...,60.0,45.0,41.0,34.0,36.0,24.0,19.0,29.0,21.0,23.0,31.0,26.0,42.0,57.0,63.0,56.0,65,56.0,47.0,59.0,58.0,55.0,48.0,47.0,44.0,36.0,37.0,40.0,39.0,31.0,22.0,17.0,13.0,17.0,31.0,37.0,46.0,48.0,38.0,54.0
2019-01-03 13:00:00,48.0,36.0,55.0,48.0,27.0,47.0,46.0,58.0,51.0,42.0,50.0,62.0,44.0,47.0,58.0,44.0,53.0,44.0,43.0,44.0,38.0,45.0,35.0,34.0,38.0,26.0,23.0,17.0,21.0,27.0,39.0,37.0,53.0,67.0,50.0,69.0,59.0,66.0,50.0,70.0,...,45.0,41.0,34.0,36.0,24.0,19.0,29.0,21.0,23.0,31.0,26.0,42.0,57.0,63.0,56.0,65.0,56,47.0,59.0,58.0,55.0,48.0,47.0,44.0,36.0,37.0,40.0,39.0,31.0,22.0,17.0,13.0,17.0,31.0,37.0,46.0,48.0,38.0,54.0,68.0
2019-01-03 14:00:00,36.0,55.0,48.0,27.0,47.0,46.0,58.0,51.0,42.0,50.0,62.0,44.0,47.0,58.0,44.0,53.0,44.0,43.0,44.0,38.0,45.0,35.0,34.0,38.0,26.0,23.0,17.0,21.0,27.0,39.0,37.0,53.0,67.0,50.0,69.0,59.0,66.0,50.0,70.0,51.0,...,41.0,34.0,36.0,24.0,19.0,29.0,21.0,23.0,31.0,26.0,42.0,57.0,63.0,56.0,65.0,56.0,47,59.0,58.0,55.0,48.0,47.0,44.0,36.0,37.0,40.0,39.0,31.0,22.0,17.0,13.0,17.0,31.0,37.0,46.0,48.0,38.0,54.0,68.0,51.0
2019-01-03 15:00:00,55.0,48.0,27.0,47.0,46.0,58.0,51.0,42.0,50.0,62.0,44.0,47.0,58.0,44.0,53.0,44.0,43.0,44.0,38.0,45.0,35.0,34.0,38.0,26.0,23.0,17.0,21.0,27.0,39.0,37.0,53.0,67.0,50.0,69.0,59.0,66.0,50.0,70.0,51.0,55.0,...,34.0,36.0,24.0,19.0,29.0,21.0,23.0,31.0,26.0,42.0,57.0,63.0,56.0,65.0,56.0,47.0,59,58.0,55.0,48.0,47.0,44.0,36.0,37.0,40.0,39.0,31.0,22.0,17.0,13.0,17.0,31.0,37.0,46.0,48.0,38.0,54.0,68.0,51.0,55.0
2019-01-03 16:00:00,48.0,27.0,47.0,46.0,58.0,51.0,42.0,50.0,62.0,44.0,47.0,58.0,44.0,53.0,44.0,43.0,44.0,38.0,45.0,35.0,34.0,38.0,26.0,23.0,17.0,21.0,27.0,39.0,37.0,53.0,67.0,50.0,69.0,59.0,66.0,50.0,70.0,51.0,55.0,53.0,...,36.0,24.0,19.0,29.0,21.0,23.0,31.0,26.0,42.0,57.0,63.0,56.0,65.0,56.0,47.0,59.0,58,55.0,48.0,47.0,44.0,36.0,37.0,40.0,39.0,31.0,22.0,17.0,13.0,17.0,31.0,37.0,46.0,48.0,38.0,54.0,68.0,51.0,55.0,56.0


In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8293, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8293, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


$\textbf{When Should You Use Normalization And Standardization:}$

Normalization is a good technique to use when you do not know the distribution of your data or when you know the distribution is not Gaussian (a bell curve). Normalization is useful when your data has varying scales and the algorithm you are using does not make assumptions about the distribution of your data, such as k-nearest neighbors and artificial neural networks.

In [ ]:
## normalizing the data

# MinMaxScaler() transforms features by scaling each feature to a given range (given by feature_range())
# The cost of having this bounded range is that we will end up with smaller standard deviations, which can 
# suppress the effect of outliers. Thus MinMax Scalar is sensitive to outliers

#scaler = MinMaxScaler(feature_range=(0, 1))
#df_x = df.iloc[:,0:24]
# the method fit_transform() computes the min and the max used for scaling and then carries out the transformation
#df_x_scaled = scaler.fit_transform(df_x)
# later, inverse_transform() can be used to undo the scaling to the feature_range

# normalizing the entire dataset
#df_normalized = scaler.fit_transform(df)

# Building the FFNN neural network model


# Method 1 (Keras)

Tensorflow 2.0 Impelementation <br>
(tf.keras is TensorFlow's implementation of the Keras API specification. This is a high-level API to build and train models that includes first-class support for TensorFlow-specific functionality)


TF 2 keras RNN tutorial
https://www.tensorflow.org/guide/keras/rnn <br>
TF 2 time series forecasting tutorial
https://www.tensorflow.org/tutorials/structured_data/time_series

In [ ]:
# configuring the inputs for the model
# For Keras, the input has to be in the shape (samples, time steps, features)
# 24 timestep with n features where n is equal to the shape of column [1] of X_train or X_test

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
X_train.shape

(589, 1, 60)

In [ ]:
X_test.shape

(1, 1, 60)

In [ ]:
# creating a leaky_relu activation function

def my_leaky_relu(x):
    return tf.nn.leaky_relu(x)

## Gridsearch CV for the optimal hyperparameters

Guide to Hyperparameter tuning: <br>
https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
Dropout regularization for RNNs: <br>
https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/



## One step Gridsearch CV for all hyperparameters

**Blogpost Hyperparametertuning LSTM/GRU:**

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

**Overview of Gradient Descent Algorithms:**

https://ruder.io/optimizing-gradient-descent/index.html#adagrad

In [ ]:
# creating the parameter grid as a dictionary

## Fixed hyperparameters ##
batch_size = [100]
epochs = [100]
neurons = [500]
dropout = [0.0]
learning_rate = [0.01]
optimizer = ['Adam']


param_grid_cv = dict(batch_size=batch_size, epochs=epochs, neurons=neurons, dropout = dropout, optimizer=optimizer, learning_rate = learning_rate)

## Fitting the FFNN model with fixed hyperparameters and 3 Dense (fully connected layers)


In [ ]:


# fitting the FFNN model with fixed hyperparameters
FFNN_model = tf.keras.Sequential()


FFNN_model.add(tf.keras.layers.Dense(units=128, input_shape = (1,60)))
FFNN_model.add(tf.keras.layers.Dense(units=64))


# output layer 
FFNN_model.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
FFNN_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])
FFNN_model.fit(X_train, Y_train, epochs=100, batch_size=100, verbose=1)
    


Epoch 1/100
83/83 [==============================] - 0s 2ms/step - loss: 444.6491 - mean_squared_error: 444.6491
Epoch 2/100
83/83 [==============================] - 0s 2ms/step - loss: 78.4706 - mean_squared_error: 78.4706
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 71.7512 - mean_squared_error: 71.7512
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 68.9997 - mean_squared_error: 68.9997
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 68.7532 - mean_squared_error: 68.7532
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 67.8625 - mean_squared_error: 67.8625
Epoch 7/100
83/83 [==============================] - 0s 2ms/step - loss: 67.6588 - mean_squared_error: 67.6588
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 67.5710 - mean_squared_error: 67.5710
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 67.2204 - mean_squared_error: 67.2204

In [ ]:
FFNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            7808      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 24)             1560      
Total params: 17,624
Trainable params: 17,624
Non-trainable params: 0
_________________________________________________________________


Deep GRU network (input layer - 5 stacked GRU layers - output layer)

## Undoing the normalization, making predictions and computing the test error

In [ ]:
# making predictions (regular FFNN)
y_train_pred = FFNN_model.predict(X_train)
y_test_pred = FFNN_model.predict(X_test)

In [ ]:
# MSE FFNN forecast
mse_FFNN = mean_squared_error(Y_test, y_test_pred)
mse_FFNN

56.39049682786541

In [ ]:
# RMSE FFNN forecast
rmse_FFNN = sqrt(mse_FFNN)
rmse_FFNN

7.509360613784999

In [ ]:
# mae FFNN
mae_FFNN = mean_absolute_error(Y_test, y_test_pred)
mae_FFNN

6.23969825108846

In [ ]:
# custom MAPE function
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true = Y_test
y_pred = y_test_pred

In [ ]:
# MAPE FFNN
mean_absolute_percentage_error(y_true,y_pred)

19.406129829382582

## Prediction intervals using quantile regression

In [ ]:
# quantile regression loss = tilted loss = pinball loss
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

## 2 separate PI networks

In [ ]:
## build lower bound with different loss (95% PI)
q = 0.025



# fitting the FFNN model with fixed hyperparameters
lower_FFNN = tf.keras.Sequential()


lower_FFNN.add(tf.keras.layers.Dense(units=500, input_shape = (1,60)))
lower_FFNN.add(tf.keras.layers.Dense(units=250))


# output layer 
lower_FFNN.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
lower_FFNN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=lambda y,f: tilted_loss(q,y,f))
lower_FFNN.fit(X_train, Y_train, epochs=300, batch_size=300, verbose=1)


Epoch 1/300
28/28 [==============================] - 0s 2ms/step - loss: 1.5303
Epoch 2/300
28/28 [==============================] - 0s 2ms/step - loss: 0.6401
Epoch 3/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4952
Epoch 4/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4734
Epoch 5/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4643
Epoch 6/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4594
Epoch 7/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4546
Epoch 8/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4501
Epoch 9/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4473
Epoch 10/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4521
Epoch 11/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4445
Epoch 12/300
28/28 [==============================] - 0s 2ms/step - loss: 0.4500
Epoch 13/300
28/28 [=================

In [ ]:
## build upper bound with different loss (95% PI)
q = 0.975


# fitting the FFNN model with fixed hyperparameters
upper_FFNN = tf.keras.Sequential()


upper_FFNN.add(tf.keras.layers.Dense(units=500, input_shape = (1,60)))
upper_FFNN.add(tf.keras.layers.Dense(units=250))


# output layer 
upper_FFNN.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
upper_FFNN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=lambda y,f: tilted_loss(q,y,f))
upper_FFNN.fit(X_train, Y_train, epochs=300, batch_size=300, verbose=1)

Epoch 1/300
28/28 [==============================] - 0s 2ms/step - loss: 4.5126
Epoch 2/300
28/28 [==============================] - 0s 2ms/step - loss: 0.8856
Epoch 3/300
28/28 [==============================] - 0s 2ms/step - loss: 0.6259
Epoch 4/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5794
Epoch 5/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5726
Epoch 6/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5534
Epoch 7/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5497
Epoch 8/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5557
Epoch 9/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5408
Epoch 10/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5406
Epoch 11/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5455
Epoch 12/300
28/28 [==============================] - 0s 2ms/step - loss: 0.5342
Epoch 13/300
28/28 [=================

In [ ]:
# predictions from PI models

lower_FFNN = lower_FFNN.predict(X_test)
upper_FFNN = upper_FFNN.predict(X_test)

PI measure calculations

In [ ]:
def PCIP(upper, lower, test_set):
    n = len(Y_test[0,:])
    count = 0
    PCIP = 0
    
    for i in range(n):
        if (upper[0,i] > test_set[0,i] and lower[0,i] < test_set[0,i]):
            count = count + 1 
            
    PCIP = count/n
    return PCIP

In [ ]:
# FFFNN PI measures 
PCIP(upper = upper_FFNN, lower = lower_FFNN, test_set = Y_test)

1.0

In [ ]:
def MPIW(upper, lower):
    n = len(upper[0,:])
    diff = 0
    MPIW = 0
    
    for i in range(n):
        diff = diff + (upper[0,i] - lower[0,i])
    
    MPIW = diff/n
    return MPIW

In [ ]:
MPIW(upper_FFNN, lower_FFNN)

33.48849376042684